In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
from sys import exc_info
from caiman import movie as cmovie

In [2]:
%load_ext autoreload
%autoreload 1
%aimport Regions, physio_def_1, numeric, Automatic

from physio_def_1 import *
from Regions import Regions
from numeric import *
from Automatic import *

In [3]:
os.chdir(os.path.expanduser("~"))

In [4]:
exampleNpzFile = "local_data/testdir/Experiment38a_7_Series016.npz"
# exampleNpzFile = "local_data/Sandra/2019_10_16/Experiment39a/Experiment39a_13_Series024.npz"
# exampleNpzFile = "local_data/Sandra/2019_10_16/Experiment39a/Experiment39a_8_Series019.npz"

In [5]:
outputStyle = {
    "color":"blue",
    "font-family":"Courier New",
    "font-size":"80%",
    "max-width": "700px",
    }
infoStyle = {
    "font-size":"70%",
    "font-family":"Arial",
    # "background-color":"lightcyan",
    "color":"grey",
    "max-width": "700px",
    }
bodyStyle = {
    "font-family":"Arial",
    "max-width": "700px",
    }

In [6]:
## global vars
npzFile = None
metadata = None
movie = None
regions = None
test = False

In [7]:
app = JupyterDash(__name__,
                  width=800,
#                   height=400,
                 )

APP_LAYOUT = [
    
    html.H1(children='CaRec Viewer',style=bodyStyle),
    html.Div("Please, think of a better name and let me know.",style=infoStyle),
    html.Br(),
    ####################################
#     html.H2(children='Importing',style=bodyStyle),
    html.Div('Enter path to an npz file, choose whether to check for movement, and press enter.',style={**bodyStyle, "display":"inline-box"}),
    html.Div([
        dcc.Checklist(id="check_movement_choice",options=[{"label":'Check for movement',"value":"yes"}],value=[],
                          style={**infoStyle, "display":"inline-box","float":"right"}),
    ],style={"width":"200px"}),
    
    dcc.Input(id="npz_filename",
            type="text",
            placeholder=exampleNpzFile,
            debounce=True,
            size = 100,
            value=exampleNpzFile if test else "",
        ),

#     html.Div('Loading can take a few seconds [for very large files even a minute].',style={**infoStyle,"text-align": "center"}),
    html.Br(),
    
    html.Div(id="npz_feedback", children="",style=outputStyle),#To start, enter the full path to a file and press enter. Note, that loading can take a few seconds [for very large files even a minute].

    html.Div(children=[html.Br(),"    Spatial filtering**"],style={**bodyStyle,'display': 'inline-block'}),

    dcc.Input(id="gSig_filt_input",
        type="text",
        debounce=True,
        size = 5,
        value="",
        placeholder="3",
        style={'display': 'inline-block'}),    

    html.Div(html.Abbr(
        children="[hover mouse for more info]",
        title="""For the proper analysis, you need to set the size of the spatial filter. Its value should be of the order of the  half of the typical cell dimension (in pixels!). """#In many of our recording, pixel size is around 2 µm. If we consider a typical cell of 10 µm, filter should be around 10/2/2 = 2.5. It needs to be an integer, so reasonable numbers to put would be 2 or 3, perhaps even 4. You can change it later.
        ),style={**infoStyle,'display': 'inline-block'}),

    html.Div(id="roi_feedback", children="",style=outputStyle),
    
    dcc.Graph(id='images',figure=baseFig,),

    html.Div(children="Resample*",style={'display': 'inline-block'}),
    
    dcc.Input(id="resample_input",
        type="text",
        debounce=True,
        size = 5,
        placeholder="5",
        value="",
        style={'display': 'inline-block'}),
    
    html.Div(children=" ",style={"width":"20px",'display': 'inline-block'}),
    html.Div(children=" ", id="resample_feedback",
             style={'display': 'inline-block',**outputStyle}),
    
    html.Div(style=infoStyle,
        children=[
        """
        *Resampling is optional (and irreversible). It is only useful to speed up the analysis downstream.
        Enter the new desired frequency in Hz (e.g. 5), and press enter.
        """,
        html.Br(),],),
    
    html.H3(children='ROIs', style=bodyStyle),
    html.Div("Please, choose which ROIs you'd want to see.", style=bodyStyle),
    html.Br(),
    html.Div("First, toggle some basic filtering. Then, you can use the lasso (or box) tool in the left graph to select ROIs you want to see the trace of. Use shift to select multiple regions.",style=infoStyle),
    html.Br(),
    html.Div([
        dcc.RadioItems(id="roi_choice",
            options=[
                {'label': 'Largest', 'value': 'largest'},
                {'label': 'Random', 'value': 'rnd'},
                {'label': 'Interesting', 'value': 'interest'},
                {'label': 'Central', 'value': 'central'},
            ],
         value='interest'),
        html.Div("How many?", style={**bodyStyle,"display":"inline-block"}),
        dcc.Input(id="roi_choice_number",
                type="text",
                debounce=True,
                size = 10,
                value='3' if test else "20"
            )
    ],style={"display":"inline-block","width":"49%"}),
#     html.Br(),
    html.Div([
        "Show downsampled to",
        dcc.Input(id="show_freq_input",
            type="text",
            debounce=False,
            size = 3,
            value='2'),
        "Hz"
        ], style={**bodyStyle,"display":"inline-block", "width":"49%"}, ),
    html.Br(),
    html.Div(dcc.Graph(id='show_rois',      figure=baseFig), style={"display":"inline-block"}),
    html.Div(dcc.Graph(id='show_raw_traces',figure=baseFig), style={"display":"inline-block"}),
    html.Div([
       "Trace ROIs:",
    dcc.Input(id="selected-indices",
        type="text",
        debounce=False,
        size = 60,
        value="",
         ), 
    ],style={**bodyStyle,"display":"inline-box"}),
    
    html.Div([
        html.Pre(id='selected-data',
                 style={
            'border': 'thin lightgrey solid',
            'overflowX': 'scroll',
            'overflowY': 'scroll',
            "height": "3px",
            "width": "30px",
            }),
        ],),
    html.Div(
        id="shown_rois",
        children="", style={**outputStyle,
                            "width":"10px",
                            "height": "3px",
                            'overflowX': 'scroll',
                            'overflowY': 'scroll',},),
    html.Div(
        id="spare_out",
        children="", style={**outputStyle,"width":"10px"}),

    ]

if test: test=False

In [8]:
@app.callback(
    Output("resample_feedback", "children"),
    [Input("resample_input", "value")]
             )
def resample_callback(val):
    try:
        from Automatic import resample
        if val == "":
            feedback = ""
        else:
            movie = resample(movie,newFreq)
            feedback = f"Resampling finished. New frequency is {movie.fr:.4} Hz."
            regions.update(movie)
        return feedback
    except:
        return ("ERROR: "+exc_info().__repr__(),None)[0]
    

In [9]:
@app.callback([Output("npz_feedback", "children"),
               Output("gSig_filt_input", "value")],
              [Input("npz_filename", "value")],
              [State("check_movement_choice","value")])
def loadMovie(val,mv):
    if str(val)=="":
        return ('Loading can take a few seconds [for very large files even a minute].',"")
    try:
        from pandas import read_csv
        global npzFile, movie, metadata, cell_halfwidth_in_px, regions
        npzFile = val
        metadata = read_csv(npzFile.replace("npz","txt")).loc[0]
        if not hasattr(metadata,"freq"):
            metadata.freq = 1
        movie = import_npz_files([npzFile])
        movie = movie.astype("float")
        if len(movie.shape)==2:        
            movie = movie.reshape((1,-1))
        movie = cmovie(movie)
        movie.fr = metadata.freq
        timeframes = metadata['T']
        freq = metadata.freq
        pxsize = metadata.pxSize
        feedback = [
            f"{npzFile} loaded successfully.",
            html.Br(),
            f"It has {timeframes} timeframes and is done at {freq:.4} Hz, with a pixel size of {pxsize:.3} µm.",
            html.Br(),
        ]
        if metadata.freq>5:
            
            freqMC = 3
            n_rebin = int(movie.fr/freqMC)

            rebinned_movie = rebin(movie,n_rebin).astype("float32")
            rebinned_movie = cmovie(rebinned_movie)
            rebinned_movie.fr = movie.fr/n_rebin
        else:
            rebinned_movie = movie.astype("float32")
            feedback += [
                "(This is probably a lot higher frequency than you need, consider resampling to lower frequency, e.g. 5 Hz.)",
                html.Br(),
                        ]
        cell_halfwidth_in_px = int(np.ceil(6./metadata.pxSize))
        if len(mv):
            rigidShifts = getRigidShifts(rebinned_movie, (cell_halfwidth_in_px,)*2)
            maxshift = np.abs(rigidShifts).max()
            if maxshift>cell_halfwidth_in_px/5:
                feedback += [
                    f"It seems there is also some movement in the recording, at least {maxshift}px. You may want to first correct for that.",
                    html.Br(),
                ]
        regions = createRegions(rebinned_movie, cell_halfwidth_in_px)
        regions.update(movie)
        feedback += [
            f"Automatic size of the spatial filter: {cell_halfwidth_in_px} [you can change it below]",
            html.Br(),]
#         roifeedback = f"Number of detected ROIs: {len(regions.df)}",
        
#         return feedback
#     except:
#         return html.P("ERROR: "+exc_info()[1].__repr__())
        return (feedback, str(cell_halfwidth_in_px))
    except:
        return ([html.P("ERROR: "+exc_info().__repr__())],"None")

In [10]:
@app.callback(
    [Output("roi_feedback", "children"), Output("images", "figure"), Output("roi_choice","value")],
    [Input("gSig_filt_input", "value")])
def recalcRois(val):
    if val == "":
        return ("",baseFig,"")
    try:
        global movie,regions
        x = int(val)
        if regions.filterSize != x:
            regions = createRegions(movie, x)
            feedback = [f"Size of the spatial filter changed to {x}.",html.Br()]
        else:
            feedback = [""]
        feedback += [f" Number of detected ROIs: {len(regions.df)}"]
        return (feedback, plotStatsFig(regions),"interest")
    except:
        return ("ERROR from recalcRois: "+exc_info().__repr__(),go.Figure(),"")

In [11]:
@app.callback(
    [Output("shown_rois", "children"),
     Output("show_rois", "figure"),
    ],
    [Input("roi_choice", "value")],
    [State("roi_choice_number", "value")],)
def parseRoiChoices_callback(roi_mode,roi_number):
    try:
        if roi_mode=="":
            return "", baseFig
        else:
            global regions
            ix = parseRoiChoices(regions, roi_mode,roi_number)
            feedback = ", ".join(ix.astype(str))
            trfig = showRoisOnly(regions,indices=ix)
            return feedback,trfig
    except:
        return ("ERROR from parseRoiChoices_callback: "+exc_info().__repr__(), baseFig)

In [12]:
@app.callback(
    [Output('selected-data', 'children'),
     Output('selected-indices', 'value')],
    [Input('show_rois', 'selectedData'),
     Input("roi_choice", "value")],
    [State("roi_choice_number", "value"),]
)
def display_selected_data(selectedData,roi_mode,roi_number):
    try:
        if selectedData is None:
            selectedData = {"points":[]}
        import json
        chosen = np.unique([p["hovertext"] for p in selectedData["points"]])
        if len(chosen) ==0:
            chosen = parseRoiChoices(regions, roi_mode,roi_number).astype(str)
        return json.dumps(selectedData, indent=2) , ", ".join(chosen)
    except:
        return "ERROR from display_selected_data: "+exc_info().__repr__() , ""

In [13]:
@app.callback(
     [Output("show_raw_traces", "figure"),Output("spare_out", "children")],
    [Input("selected-indices", "value")],
    [State("show_freq_input", "value"),
    ],)
def plotRawTraces_callback(chosen, showFreq):
    try:
        global regions
        if not hasattr(regions,"time"):
            regions.calcTraces(movie)
        showFreq = float(showFreq)
        n_Avg = int(regions.movie.fr/showFreq)
        try:
            ix = np.array(chosen.strip(" ,").split(","))
            ix = ix.astype(int)
            trfig = plotRawTraces(regions,indices=ix,nAvg=n_Avg)
#             trfig.update_layout({"width":200})
            return trfig,"ok"
        except:
            return baseFig,"nothing yet"
    except:
        return baseFig, "ERROR: "+exc_info().__repr__() 

In [14]:
# app.layout = html.Div(children=APP_LAYOUT)
# app

In [15]:
app.layout = html.Div(children=APP_LAYOUT)
app._repr_html_() 
link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
HTML(f'click to start: <a href="{link2app}">{link2app}</a>')

In [34]:
exampleNpzFile

'local_data/testdir/Experiment38a_7_Series016.npz'